In [1]:
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.config_tickers import DOW_30_TICKER

In [2]:
portfolio_raw_df = YahooDownloader(start_date='2011-01-01', end_date='2022-12-31',
                                   ticker_list=DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (88535, 8)


In [3]:
from finrl.meta.preprocessor.preprocessors import FeatureEngineer

feature_engineer = FeatureEngineer(
    use_technical_indicator=True,
    use_turbulence=False,
    user_defined_feature=False
)

df = feature_engineer.preprocess_data(portfolio_raw_df)
df

Successfully added technical indicators


,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2011-01-03,9.917948,11.795000,11.601429,11.630000,445138400,AAPL,0,0.000000,10.017025,9.870629,100.000000,66.666667,100.000000,9.917948,9.917948
3020,2011-01-03,38.752167,56.279999,55.180000,55.200001,5453300,AMGN,0,0.000000,10.017025,9.870629,100.000000,66.666667,100.000000,38.752167,38.752167
6040,2011-01-03,35.330551,43.619999,43.110001,43.299999,7633300,AXP,0,0.000000,10.017025,9.870629,100.000000,66.666667,100.000000,35.330551,35.330551
9060,2011-01-03,53.034515,66.680000,66.000000,66.150002,8072900,BA,0,0.000000,10.017025,9.870629,100.000000,66.666667,100.000000,53.034515,53.034515
12080,2011-01-03,65.527557,94.809998,94.110001,94.379997,5231500,CAT,0,0.000000,10.017025,9.870629,100.000000,66.666667,100.000000,65.527557,65.527557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75499,2022-12-30,512.076172,530.500000,524.840027,530.000000,1849600,UNH,4,-0.693236,529.770315,500.275779,50.390130,-34.013294,0.949572,513.389748,510.954844
78519,2022-12-30,204.215942,208.039993,205.699997,206.309998,4159400,V,4,-0.171918,212.501551,198.049571,52.113766,-56.217997,0.442858,206.255225,198.875282
81539,2022-12-30,33.277489,39.689999,39.070000,39.310001,44007200,VZ,4,0.260706,33.362616,30.590960,53.943995,130.020785,28.270982,32.195259,31.748041
84559,2022-12-30,32.217491,37.380001,36.970001,37.240002,5406200,WBA,4,-0.398673,36.474667,31.806406,46.462091,-171.163792,26.863815,34.556766,32.465041


In [4]:
import pandas as pd

# add covariance matrix as states
df=df.sort_values(['date','tic'],ignore_index=True)
df.index = df.date.factorize()[0]

cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df.index.unique())):
  data_lookback = df.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values
  cov_list.append(covs)


df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df = df.merge(df_cov, on='date')
df = df.sort_values(['date','tic']).reset_index(drop=True)

In [5]:
from finrl.meta.preprocessor.preprocessors import data_split

train = data_split(df, "2011-01-01", "2020-01-01")
train

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2012-01-03,12.375391,14.732143,14.607143,14.621429,302220800,AAPL,1,0.164204,12.432536,11.296688,57.530720,151.566739,32.770714,11.681643,11.828289,"[[0.0002726666551779453, 8.386800104631264e-05...",tic AAPL AMGN AXP ...
0,2012-01-03,45.188396,65.190002,63.450001,64.949997,10216800,AMGN,1,1.400030,46.873645,38.736924,64.687049,123.366427,33.146598,41.737426,40.874434,"[[0.0002726666551779453, 8.386800104631264e-05...",tic AAPL AMGN AXP ...
0,2012-01-03,40.022007,48.959999,48.139999,48.299999,6955400,AXP,1,-0.043623,40.762759,38.099377,51.463789,95.757855,9.417737,39.097620,39.659064,"[[0.0002726666551779453, 8.386800104631264e-05...",tic AAPL AMGN AXP ...
0,2012-01-03,60.731060,75.000000,74.120003,74.699997,6859300,BA,1,1.428186,61.638843,56.327738,59.259326,105.149978,37.149880,57.425600,55.191451,"[[0.0002726666551779453, 8.386800104631264e-05...",tic AAPL AMGN AXP ...
0,2012-01-03,66.636070,95.110001,92.769997,92.769997,8177000,CAT,1,-0.104179,69.229860,60.770703,53.066273,53.858899,6.804549,65.137362,64.383037,"[[0.0002726666551779453, 8.386800104631264e-05...",tic AAPL AMGN AXP ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,2019-12-31,271.987885,294.089996,291.149994,293.660004,2224200,UNH,1,6.686678,279.657238,252.655509,68.470144,81.098765,38.929012,262.855344,244.140971,"[[0.00027111650229894415, 6.0548836725056805e-...",tic AAPL AMGN AXP ...
2011,2019-12-31,181.074371,188.000000,186.539993,187.059998,5273000,V,1,1.999852,183.775001,172.944439,58.615922,96.584009,12.589647,177.293574,173.936899,"[[0.00027111650229894415, 6.0548836725056805e-...",tic AAPL AMGN AXP ...
2011,2019-12-31,45.064518,61.400002,60.939999,61.209999,8466700,VZ,1,0.240741,45.460486,44.267802,56.217687,58.313051,8.866464,44.523590,44.299424,"[[0.00027111650229894415, 6.0548836725056805e-...",tic AAPL AMGN AXP ...
2011,2019-12-31,44.828381,59.470001,58.810001,58.820000,3305100,WBA,1,0.000825,45.323667,43.493688,54.048394,-6.144214,16.194864,44.944702,43.885513,"[[0.00027111650229894415, 6.0548836725056805e-...",tic AAPL AMGN AXP ...


In [6]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step


    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]


    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique())-1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values)*weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value


        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state

    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output


    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [7]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
tech_indicator_list = ['macd', 'rsi_30', 'cci_30', 'dx_30']
feature_dimension = len(tech_indicator_list)
print(f"Feature Dimension: {feature_dimension}")

Stock Dimension: 29, State Space: 29
Feature Dimension: 4


In [8]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 10_000,
    "transaction_cost_pct": 0,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": tech_indicator_list,
    "action_space": stock_dimension,
    "reward_scaling": 1

}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [9]:
env_train, _ = e_train_gym.get_sb_env()
env_train

/home/advin4603/IIITCourseWork/AO/Project/.venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [10]:
from finrl.agents.stablebaselines3.models import DRLAgent
import torch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

agent = DRLAgent(env=env_train)

model_ddpg = agent.get_model(model_name="sac", model_kwargs={"batch_size": 4096, "buffer_size": 10000, "learning_rate": 3e-4,"learning_starts": 100, "device": device, "action_noise": "normal"})
model_ddpg

{'batch_size': 4096, 'buffer_size': 10000, 'learning_rate': 0.0003, 'learning_starts': 100, 'device': 'cuda:0', 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cuda:0 device


In [11]:
trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name="ddpg", total_timesteps=20_000)

begin_total_asset:10000
end_total_asset:36104.54368768714
Sharpe:  1.3141520845712704
begin_total_asset:10000
end_total_asset:35271.07072207674
Sharpe:  1.287172820923952
begin_total_asset:10000
end_total_asset:36827.55702008415
Sharpe:  1.3258049425335747
begin_total_asset:10000
end_total_asset:38468.73031949483
Sharpe:  1.3616906807854183
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 29        |
|    time_elapsed    | 277       |
|    total_timesteps | 8048      |
| train/             |           |
|    actor_loss      | -6.31e+05 |
|    critic_loss     | 2.65e+08  |
|    ent_coef        | 9.78      |
|    ent_coef_loss   | -118      |
|    learning_rate   | 0.0003    |
|    n_updates       | 7947      |
|    reward          | 38468.73  |
----------------------------------
begin_total_asset:10000
end_total_asset:38454.288786264704
Sharpe:  1.3666596125945563
begin_total_asset:10000
end_total_asset:37115

In [12]:
test = data_split(df, "2020-01-01", "2020-12-31")
e_test_gym = StockPortfolioEnv(df = test, **env_kwargs)
env_test = e_test_gym.get_sb_env()

DRLAgent.DRL_prediction(trained_ddpg, e_test_gym)

/home/advin4603/IIITCourseWork/AO/Project/.venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:10000
end_total_asset:10794.403746911172
Sharpe:  0.3919463378075992
hit end!


(           date  daily_return
 0    2020-01-02      0.000000
 1    2020-01-03     -0.008006
 2    2020-01-06      0.002697
 3    2020-01-07     -0.005105
 4    2020-01-08      0.002131
 ..          ...           ...
 247  2020-12-23      0.005570
 248  2020-12-24      0.001863
 249  2020-12-28      0.006499
 250  2020-12-29     -0.001025
 251  2020-12-30      0.000727
 
 [252 rows x 2 columns],
                 AAPL      AMGN       AXP        BA       CAT       CRM  \
 date                                                                     
 2020-01-02  0.034483  0.034483  0.034483  0.034483  0.034483  0.034483   
 2020-01-03  0.046269  0.047704  0.049158  0.048810  0.021994  0.022495   
 2020-01-06  0.044456  0.047724  0.048985  0.048624  0.023248  0.019759   
 2020-01-07  0.047059  0.047943  0.048809  0.048515  0.022604  0.018806   
 2020-01-08  0.048648  0.046723  0.048266  0.047921  0.018665  0.018287   
 ...              ...       ...       ...       ...       ...       ...   
 

In [13]:
test = data_split(df, "2021-01-01", "2021-12-31")
e_test_gym = StockPortfolioEnv(df = test, **env_kwargs)
env_test = e_test_gym.get_sb_env()

DRLAgent.DRL_prediction(trained_ddpg, e_test_gym)

/home/advin4603/IIITCourseWork/AO/Project/.venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:10000
end_total_asset:12381.745136290705
Sharpe:  1.8001166962901562
hit end!


(           date  daily_return
 0    2021-01-04      0.000000
 1    2021-01-05      0.006531
 2    2021-01-06      0.017100
 3    2021-01-07      0.008976
 4    2021-01-08      0.001762
 ..          ...           ...
 246  2021-12-23      0.005133
 247  2021-12-27      0.009403
 248  2021-12-28      0.003159
 249  2021-12-29      0.002617
 250  2021-12-30     -0.002621
 
 [251 rows x 2 columns],
                 AAPL      AMGN       AXP        BA       CAT       CRM  \
 date                                                                     
 2021-01-04  0.034483  0.034483  0.034483  0.034483  0.034483  0.034483   
 2021-01-05  0.030223  0.048838  0.050048  0.049654  0.032752  0.020495   
 2021-01-06  0.026395  0.048568  0.049894  0.049474  0.034187  0.021192   
 2021-01-07  0.030098  0.046914  0.049433  0.048892  0.025007  0.019095   
 2021-01-08  0.019447  0.048261  0.050334  0.049990  0.039958  0.019988   
 ...              ...       ...       ...       ...       ...       ...   
 

In [14]:
test = data_split(df, "2022-01-01", "2022-12-31")
e_test_gym = StockPortfolioEnv(df = test, **env_kwargs)
env_test = e_test_gym.get_sb_env()

DRLAgent.DRL_prediction(trained_ddpg, e_test_gym)

/home/advin4603/IIITCourseWork/AO/Project/.venv/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:10000
end_total_asset:9149.81029517336
Sharpe:  -0.3347136326102168
hit end!


(           date  daily_return
 0    2022-01-03      0.000000
 1    2022-01-04      0.007688
 2    2022-01-05     -0.006746
 3    2022-01-06     -0.004195
 4    2022-01-07      0.003008
 ..          ...           ...
 246  2022-12-23      0.004813
 247  2022-12-27      0.001436
 248  2022-12-28     -0.012297
 249  2022-12-29      0.011687
 250  2022-12-30     -0.000907
 
 [251 rows x 2 columns],
                 AAPL      AMGN       AXP        BA       CAT       CRM  \
 date                                                                     
 2022-01-03  0.034483  0.034483  0.034483  0.034483  0.034483  0.034483   
 2022-01-04  0.047004  0.046878  0.048253  0.047922  0.019351  0.032724   
 2022-01-05  0.047895  0.047322  0.049324  0.048932  0.021823  0.029680   
 2022-01-06  0.047539  0.045751  0.048206  0.047592  0.020071  0.019965   
 2022-01-07  0.043365  0.046343  0.047854  0.047313  0.024153  0.022646   
 ...              ...       ...       ...       ...       ...       ...   
 